<a href="https://colab.research.google.com/github/hemanth-sunkireddy/kaggle-iiith-location-challenge/blob/main/model-v3ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -------------------------
# Mount Google Drive
# -------------------------
from google.colab import drive
drive.mount('/content/drive')

# One-time copy (run before training)
!cp -r /content/drive/MyDrive/SMAI_Project/images_train /content/
!cp -r /content/drive/MyDrive/SMAI_Project/images_val /content/



Mounted at /content/drive


In [3]:
!cp -r /content/drive/MyDrive/SMAI_Project/labels_train.csv /content/
!cp -r /content/drive/MyDrive/SMAI_Project/labels_val.csv /content/

In [ ]:
!pip install torch torchvision --force-reinstall

In [4]:
import os
import pandas as pd
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm

print("🚀 Starting the enhanced image classification pipeline...")

# -------------------------
# Set Paths
# -------------------------
base_path = '/content/drive/MyDrive/SMAI_Project'
train_csv_path = os.path.join(base_path, 'labels_train.csv')
val_csv_path = os.path.join(base_path, 'labels_val.csv')
train_img_dir = '/content/images_train'
val_img_dir = '/content/images_val'
submission_output_path = os.path.join(base_path, '2022101005_1.csv')

# -------------------------
# Dataset
# -------------------------
class RegionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        print(f"📦 Initialized dataset with {len(self.df)} samples from {img_dir}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        label = row['Region_ID'] - 1  # 0-indexed

        if self.transform:
            image = self.transform(image)

        return image, label

# -------------------------
# Transformations
# -------------------------
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# -------------------------
# Load Data
# -------------------------
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

train_dataset = RegionDataset(train_df, train_img_dir, train_transform)
val_dataset = RegionDataset(val_df, val_img_dir, val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2, pin_memory=True)

# -------------------------
# Model & Optimizer
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")

model = models.resnet50(pretrained=True)

# Fine-tune entire model
for param in model.parameters():
    param.requires_grad = True

model.fc = nn.Linear(model.fc.in_features, 15)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# -------------------------
# Training Loop
# -------------------------
epochs = 15
best_val_acc = 0

print("🚦 Starting training...")
for epoch in range(epochs):
    model.train()
    running_loss = 0
    pbar = tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/{epochs}", leave=False)

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f"{running_loss / (pbar.n + 1):.4f}"})

    avg_loss = running_loss / len(train_loader)

    # -------------------------
    # Validation Accuracy
    # -------------------------
    model.eval()
    predictions = []
    ground_truth = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(labels.numpy())

    val_acc = accuracy_score(ground_truth, predictions)
    print(f"✅ Epoch {epoch+1} | Loss: {avg_loss:.4f} | Validation Accuracy: {val_acc:.4f}")

    # Save best model (optional)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(base_path, 'best_model.pth'))

    scheduler.step()

# -------------------------
# Final Evaluation & Submission File
# -------------------------
print("🔍 Final evaluation on validation set...")

model.eval()
predictions = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

val_preds_df = pd.DataFrame({
    'id': list(range(369)),
    'Region_ID': [p + 1 for p in predictions]
})

test_df = pd.DataFrame({
    'id': list(range(369, 738)),
    'Region_ID': [1] * 369
})

submission_df = pd.concat([val_preds_df, test_df], ignore_index=True)
submission_df.to_csv(submission_output_path, index=False)
print(f"📁 Submission saved to {submission_output_path}")

print("🎉 All steps completed. Best Validation Accuracy: {:.4f}".format(best_val_acc))


🚀 Starting the enhanced image classification pipeline...
📦 Initialized dataset with 6542 samples from /content/images_train
📦 Initialized dataset with 369 samples from /content/images_val
💻 Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 60.8MB/s]


🚦 Starting training...


...

✅ Epoch 20 | Loss: 0.0056 | Validation Accuracy: 0.9295


✅ Epoch 21 | Loss: 0.0043 | Validation Accuracy: 0.9350


✅ Epoch 22 | Loss: 0.0056 | Validation Accuracy: 0.9404


✅ Epoch 23 | Loss: 0.0047 | Validation Accuracy: 0.9377


KeyboardInterrupt: 

In [5]:
import os
import pandas as pd
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm

print("🚀 Starting the enhanced image classification pipeline...")

# -------------------------
# Set Paths
# -------------------------
base_path = '/content/drive/MyDrive/SMAI_Project'
train_csv_path = os.path.join(base_path, 'labels_train.csv')
val_csv_path = os.path.join(base_path, 'labels_val.csv')
train_img_dir = '/content/images_train'
val_img_dir = '/content/images_val'
submission_output_path = os.path.join(base_path, '2022101005_1.csv')

# -------------------------
# Dataset
# -------------------------
class RegionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        print(f"📦 Initialized dataset with {len(self.df)} samples from {img_dir}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        label = row['Region_ID'] - 1  # 0-indexed

        if self.transform:
            image = self.transform(image)

        return image, label

# -------------------------
# Transformations
# -------------------------
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# -------------------------
# Load Data
# -------------------------
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

train_dataset = RegionDataset(train_df, train_img_dir, train_transform)
val_dataset = RegionDataset(val_df, val_img_dir, val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2, pin_memory=True)

# -------------------------
# Model & Optimizer
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")

model = models.resnet50(pretrained=True)

# Fine-tune entire model
for param in model.parameters():
    param.requires_grad = True

model.fc = nn.Linear(model.fc.in_features, 15)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# -------------------------
# Training Loop
# -------------------------
epochs = 100
best_val_acc = 0
best_model_state_dict = None

print("🚦 Starting training...")
for epoch in range(epochs):
    model.train()
    running_loss = 0
    pbar = tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/{epochs}", leave=False)

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f"{running_loss / (pbar.n + 1):.4f}"})

    avg_loss = running_loss / len(train_loader)

    # -------------------------
    # Validation Accuracy
    # -------------------------
    model.eval()
    predictions = []
    ground_truth = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(labels.numpy())

    val_acc = accuracy_score(ground_truth, predictions)
    print(f"✅ Epoch {epoch+1} | Loss: {avg_loss:.4f} | Validation Accuracy: {val_acc:.4f}")

    # If current validation accuracy is better, save the model and generate the CSV
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state_dict = model.state_dict()
        torch.save(model.state_dict(), os.path.join(base_path, 'best_model.pth'))

        # Generate the submission file
        print("🔄 Best model found, generating submission file...")

        # Generate predictions for submission
        model.eval()
        predictions = []
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                predictions.extend(preds.cpu().numpy())

        val_preds_df = pd.DataFrame({
            'id': list(range(369)),
            'Region_ID': [p + 1 for p in predictions]
        })

        test_df = pd.DataFrame({
            'id': list(range(369, 738)),
            'Region_ID': [1] * 369
        })

        submission_df = pd.concat([val_preds_df, test_df], ignore_index=True)
        submission_df.to_csv(submission_output_path, index=False)
        print(f"📁 Submission saved to {submission_output_path}")

    scheduler.step()

# -------------------------
# Final Evaluation
# -------------------------
print(f"🎉 Best Validation Accuracy: {best_val_acc:.4f}")


🚀 Starting the enhanced image classification pipeline...
📦 Initialized dataset with 6542 samples from /content/images_train
📦 Initialized dataset with 369 samples from /content/images_val
💻 Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🚦 Starting training...


✅ Epoch 1 | Loss: 1.4106 | Validation Accuracy: 0.7263
🔄 Best model found, generating submission file...
📁 Submission saved to /content/drive/MyDrive/SMAI_Project/2022101005_1.csv


✅ Epoch 2 | Loss: 0.5282 | Validation Accuracy: 0.8401
🔄 Best model found, generating submission file...
📁 Submission saved to /content/drive/MyDrive/SMAI_Project/2022101005_1.csv


...

✅ Epoch 20 | Loss: 0.0051 | Validation Accuracy: 0.9458
🔄 Best model found, generating submission file...
📁 Submission saved to /content/drive/MyDrive/SMAI_Project/2022101005_1.csv


✅ Epoch 21 | Loss: 0.0058 | Validation Accuracy: 0.9350


✅ Epoch 22 | Loss: 0.0063 | Validation Accuracy: 0.9377


✅ Epoch 23 | Loss: 0.0056 | Validation Accuracy: 0.9295


KeyboardInterrupt: 

In [7]:
import os
import pandas as pd
import numpy as np
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class AngleDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        angle = row['angle']
        if self.transform:
            image = self.transform(image)
        return image, angle

def mean_absolute_angular_error(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    return np.mean(np.minimum(np.abs(true - pred), 360 - np.abs(true - pred)))

base_dir = '/content'
train_csv = 'labels_train.csv'
val_csv = 'labels_val.csv'
train_img_dir = os.path.join(base_dir, 'images_train')
val_img_dir = os.path.join(base_dir, 'images_val')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_df = pd.read_csv(os.path.join(base_dir, train_csv))
val_df = pd.read_csv(os.path.join(base_dir, val_csv))

train_dataset = AngleDataset(train_df, train_img_dir, transform)
val_dataset = AngleDataset(val_df, val_img_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

best_maae = float('inf')
submission_path = '2022101005_1.csv'

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for images, angles in tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/100"):
        images = images.to(device)
        angles = angles.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, angles)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"✅ Epoch {epoch+1}: Loss = {running_loss / len(train_loader):.4f}")

    model.eval()
    val_preds = []
    val_gts = []
    with torch.no_grad():
        for images, angles in val_loader:
            images = images.to(device)
            angles = angles.numpy()
            outputs = model(images).cpu().numpy().flatten()
            val_preds.extend(outputs)
            val_gts.extend(angles)

    val_preds_clamped = [round(max(0, min(360, p))) for p in val_preds]
    maae_score = mean_absolute_angular_error(val_gts, val_preds_clamped)
    print(f"🎯 Epoch {epoch+1}: Validation MAAE = {maae_score:.2f} degrees")

    if maae_score < best_maae:
        best_maae = maae_score
        val_submission = pd.DataFrame({
            'id': list(range(len(val_preds_clamped))),
            'angle': val_preds_clamped
        })
        dummy_test_submission = pd.DataFrame({
            'id': list(range(len(val_preds_clamped), 738)),
            'angle': [0] * (738 - len(val_preds_clamped))
        })
        final_submission = pd.concat([val_submission, dummy_test_submission], ignore_index=True)
        final_submission.to_csv(submission_path, index=False)
        print(f"💾 Best MAAE improved to {best_maae:.2f}, submission saved.")


✅ Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
📚 Epoch 1/100: 100%|██████████| 205/205 [01:38<00:00,  2.07it/s]


✅ Epoch 1: Loss = 36796.2626
🎯 Epoch 1: Validation MAAE = 90.08 degrees
💾 Best MAAE improved to 90.08, submission saved.


📚 Epoch 2/100: 100%|██████████| 205/205 [01:39<00:00,  2.06it/s]


✅ Epoch 2: Loss = 25714.7889
🎯 Epoch 2: Validation MAAE = 93.15 degrees


...

✅ Epoch 8: Loss = 1213.5447
🎯 Epoch 8: Validation MAAE = 61.31 degrees
💾 Best MAAE improved to 61.31, submission saved.


📚 Epoch 9/100: 100%|██████████| 205/205 [01:39<00:00,  2.07it/s]


✅ Epoch 9: Loss = 882.7534
🎯 Epoch 9: Validation MAAE = 60.89 degrees
💾 Best MAAE improved to 60.89, submission saved.


📚 Epoch 10/100:  31%|███       | 63/205 [00:30<01:08,  2.07it/s]


KeyboardInterrupt: 

In [ ]:
# ⛰️ Colab and Environment Setup
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

# 📁 Define paths inside your Google Drive
root_dir = '/content'
train_csv = os.path.join(root_dir, 'labels_train.csv')
val_csv = os.path.join(root_dir, 'labels_val.csv')
train_img_dir = os.path.join(root_dir, 'images_train')
val_img_dir = os.path.join(root_dir, 'images_val')

# 🧾 Read the CSVs
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)

# ❌ Remove anomaly IDs from val
anomaly_ids = [95, 145, 146, 158, 159, 160, 161]
val_df_cleaned = val_df[~val_df.index.isin(anomaly_ids)].reset_index(drop=True)

# 🖼️ Custom Dataset
class GeoDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, task='both'):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.task = task

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        lat = int(row['latitude'])
        lon = int(row['longitude'])
        return image, torch.tensor([lat, lon], dtype=torch.float)

# 📦 Transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# 🔄 Dataloaders
train_dataset = GeoDataset(train_df, train_img_dir, transform)
val_dataset = GeoDataset(val_df_cleaned, val_img_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 🧠 Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# ⚙️ Loss & Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 🏋️‍♂️ Training
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"📚 Epoch {epoch+1} Loss: {running_loss / len(train_loader):.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch 1/10: 100%|██████████| 205/205 [00:45<00:00,  4.54it/s]


📚 Epoch 1 Loss: 318913120435.8244


Epoch 2/10: 100%|██████████| 205/205 [00:46<00:00,  4.45it/s]


...

📚 Epoch 8 Loss: 318856819497.2098


Epoch 9/10: 100%|██████████| 205/205 [00:45<00:00,  4.49it/s]


📚 Epoch 9 Loss: 318853855341.8927


Epoch 10/10: 100%|██████████| 205/205 [00:45<00:00,  4.49it/s]


📚 Epoch 10 Loss: 318844135863.5707
📝 Generating submission...
✅ Submission file '2022101005_1.csv' created!


In [ ]:
# ❌ Remove anomaly IDs from val and preserve original indices
anomaly_ids = [95, 145, 146, 158, 159, 160, 161]
val_df_cleaned = val_df.drop(anomaly_ids).reset_index(drop=False)  # Keep original index as 'index'
val_df_cleaned.rename(columns={'index': 'id'}, inplace=True)       # Rename index to 'id'

# ...
# 📈 Validation
model.eval()
val_preds = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.round().cpu().numpy().astype(int)
        val_preds.extend(preds)

# 📤 Submission file
print("📝 Generating submission...")
val_submission = pd.DataFrame({
    'id': val_df_cleaned['id'].tolist(),  # Use original id
    'Latitude': [lat for lat, lon in val_preds],
    'Longitude': [lon for lat, lon in val_preds]
})

# Add 0,0 for test samples
test_ids = list(range(369, 738))
test_submission = pd.DataFrame({
    'id': test_ids,
    'Latitude': [0] * len(test_ids),
    'Longitude': [0] * len(test_ids)
})

# Combine and save
submission_df = pd.concat([val_submission, test_submission], ignore_index=True)
submission_df = submission_df.sort_values(by='id').reset_index(drop=True)
submission_df.to_csv('2022101005_1.csv', index=False)
print("✅ Submission file '2022101005_1.csv' created!")


📝 Generating submission...
✅ Submission file '2022101005_1.csv' created!
